In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [2]:
diabetes = pd.read_csv('data/diabetes.csv')

In [3]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [4]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'])

In [5]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [6]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

### Train and Test with Bias

In [7]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput  = experiment_TRAIN(X_train, 
                                                                                         y_train,
                                                                                         lrParameter=0.2, 
                                                                                         n_epochs=40, 
                                                                                         trainingBias=True,
                                                                                         error_by_epoch = False,
                                                                                         trainingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input'})

In [11]:
with open('results/version4/error_by_epoch_unbiased.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [7]:
weightsEncodingWeight, weightsEncodingInput, weightsPhaseEncoding, weightsHSGS, weightsVectorsClassico, weightVectorsClassicoBin = experiment_TRAIN(X_train, 
                                                                                                             y_train,
                                                                                                             lrParameter=0.2, 
                                                                                                             n_epochs=20, 
                                                                                                             trainingBias=False,
                                                                                                             trainingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input', 'phase-encoding'})


erro HSGS 10.19921875
erro encoding weight 506.3291015625
erro encoding input 318.7802734375
erro phase encoding 175.2587890625
erro classico 88.6180606738132
erro classico Bin 212.65483554319545

erro HSGS 0.0
erro encoding weight 507.87890625
erro encoding input 318.5244140625
erro phase encoding 184.681640625
erro classico 74.28810022119933
erro classico Bin 205.94105390216203

erro HSGS 0.0
erro encoding weight 507.322265625
erro encoding input 317.830078125
erro phase encoding 183.9775390625
erro classico 74.28873214639039
erro classico Bin 205.94105390216203

erro HSGS 0.0
erro encoding weight 507.755859375
erro encoding input 317.4716796875
erro phase encoding 184.6552734375
erro classico 74.28873280953874
erro classico Bin 205.94105390216203

erro HSGS 0.0
erro encoding weight 507.814453125
erro encoding input 317.697265625
erro phase encoding 184.1142578125
erro classico 74.28873281027232
erro classico Bin 205.94105390216203

erro HSGS 0.0
erro encoding weight 507.583984375
e

In [8]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsPhaseEncoding, 
          weightsHSGS, 
          weightsVectorsClassico, 
          weightVectorsClassicoBin]

In [9]:
with open('results/version4/weights_unbiased_v4.json', 'w+') as fp:
    json.dump(weights, fp)

#### Test

In [10]:
with open('results/version4/weights_unbiased_v4.json') as json_file:
    weights = json.load(json_file)

In [ ]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              weights[4], 
                              weights[5], 
                              repeat=10,
                              bias=False, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'phase-encoding','encoding-input'})

In [ ]:
with open('results/version4/experiments_unbiased.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)

## Save results as dataframe

In [ ]:
resultsBIAS

In [21]:
def readAndResults(i):
    with open('results/'+i) as f:
        data = json.load(f)

    erros = ['error_HSGS', 'error_encoding_weight', 'error_encoding_input', 'error_classic', 'error_classic_bin']
    new_data = {'error':[], 'model':[]}

    for erro in erros:
        new_data['error'] = new_data['error'] + data[erro] 
        new_data['model'] = new_data['model'] +  ( (erro+' ') * len(data[erro])).split(' ')[:-1]  

    new_data = pd.DataFrame(new_data)

    new_data.to_csv('results/'+i[:-5]+'_dataframe.csv', index=False)
    
    return new_data

In [22]:
readAndResults('HEALTH_ZERO.json')

error              model
0    0.3646         error_HSGS
1    0.3646         error_HSGS
2    0.3646         error_HSGS
3    0.3646         error_HSGS
4    0.3646         error_HSGS
..      ...                ...
195  0.3646  error_classic_bin
196  0.3646  error_classic_bin
197  0.3646  error_classic_bin
198  0.3646  error_classic_bin
199  0.3646  error_classic_bin

[200 rows x 2 columns]

In [5]:
df = pd.read_csv('results/HEALTH_ZERO_dataframe.csv')

In [10]:
np.mean(df[df.model == 'error_encoding_weight'])

error    0.36291
dtype: float64

In [11]:
np.mean(df[df.model == 'error_encoding_input'])

error    0.480335
dtype: float64

In [12]:
df2 = pd.read_csv('results/results_withoutbias_dataframe.csv')

In [13]:
np.mean(df2[df2.model == 'error_encoding_weight'])

error    0.573043
dtype: float64

In [14]:
np.mean(df2[df2.model == 'error_encoding_input'])

error    0.493495
dtype: float64